In [9]:
from dotenv import load_dotenv
import os
import logging
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableSequence
from langchain_core.prompts.chat import HumanMessagePromptTemplate, SystemMessagePromptTemplate
from sendgrid import SendGridAPIClient
from sendgrid.helpers.mail import Mail, Email, To, Content

# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Load environment variables
load_dotenv(override=True)
groq_api_key = os.getenv("GROQ_API_KEY")
sendgrid_api_key = os.getenv("SENDGRID_API_KEY")
if not groq_api_key:
    print("Error: GROQ_API_KEY not found in environment variables.")
    exit(1)
if not sendgrid_api_key:
    print("Error: SENDGRID_API_KEY not found in environment variables.")
    exit(1)

# Initialize the LLM
llm = ChatOpenAI(
    model="llama3-70b-8192",
    api_key=groq_api_key,
    base_url="https://api.groq.com/openai/v1",
)

# Instructions for agents
instructions = {
    "agent1": """You are a sales agent working for ComplAI, a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. 
You write professional, serious cold emails. Do not give anything else in the response except the Email.""",
    "agent2": """You are a humorous, engaging sales agent working for ComplAI, a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. 
You write witty, engaging cold emails that are likely to get a response. Do not give anything else in the response except the Email.""",
    "agent3": """You are a busy sales agent working for ComplAI, a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. 
You write concise, to-the-point cold emails. Do not give anything else in the response except the Email.""",
    "sales_manager": """You are a Sales Manager at ComplAI. Your goal is to select and send the best cold sales email from the provided drafts.
Follow these steps:
1. Review the three email drafts provided, separated by '---'.
2. Select the single best email based on clarity, engagement, and likelihood of response.
3. Call the send_email tool with the selected email content.
Rules:
- Do not generate drafts; use the provided drafts.
- Call the send_email tool exactly once with the best email.
- If no draft is suitable (e.g., all contain errors), return 'No suitable email found.'
Return only the result of the send_email tool or 'No suitable email found.'"""
}

# Sales agent tools
memory_agent1 = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
@tool
def agent1_tool(input: str) -> str:
    """Professional cold email agent for ComplAI."""
    logger.info(f"Running agent1_tool with input: {input}")
    prompt = ChatPromptTemplate.from_messages([
        SystemMessagePromptTemplate.from_template(instructions["agent1"]),
        HumanMessagePromptTemplate.from_template("{input}")
    ])
    chain = RunnableSequence(prompt | llm)
    result = chain.invoke({"input": input, "chat_history": memory_agent1.buffer}).content
    logger.info(f"agent1_tool output: {result[:100]}...")
    return result

memory_agent2 = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
@tool
def agent2_tool(input: str) -> str:
    """Humorous cold email agent for ComplAI."""
    logger.info(f"Running agent2_tool with input: {input}")
    prompt = ChatPromptTemplate.from_messages([
        SystemMessagePromptTemplate.from_template(instructions["agent2"]),
        HumanMessagePromptTemplate.from_template("{input}")
    ])
    chain = RunnableSequence(prompt | llm)
    result = chain.invoke({"input": input, "chat_history": memory_agent2.buffer}).content
    logger.info(f"agent2_tool output: {result[:100]}...")
    return result

memory_agent3 = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
@tool
def agent3_tool(input: str) -> str:
    """Concise cold email agent for ComplAI."""
    logger.info(f"Running agent3_tool with input: {input}")
    prompt = ChatPromptTemplate.from_messages([
        SystemMessagePromptTemplate.from_template(instructions["agent3"]),
        HumanMessagePromptTemplate.from_template("{input}")
    ])
    chain = RunnableSequence(prompt | llm)
    result = chain.invoke({"input": input, "chat_history": memory_agent3.buffer}).content
    logger.info(f"agent3_tool output: {result[:100]}...")
    return result

# Send email tool
@tool
def send_email(email_content: str) -> str:
    """Sends the selected email to the recipient using SendGrid."""
    logger.info(f"send_email called with content: {email_content[:50]}...")
    try:
        message = Mail(
            from_email=Email("dabhideep44@gmail.com"),
            to_emails=To("dabhideep424@gmail.com"),  # Replace with actual recipient email
            subject="Cold Sales Email from ComplAI",
            plain_text_content=Content("text/plain", email_content)
        )
        sg = SendGridAPIClient(sendgrid_api_key)
        response = sg.send(message)
        logger.info("Email sent successfully")
        return f"Email sent successfully: {email_content[:50]}..."
    except Exception as e:
        logger.error(f"Error sending email: {str(e)}")
        return f"Error sending email: {str(e)}"

# Sales Manager agent
memory_sales_manager = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
def sales_manager(input_prompt: str) -> str:
    """Sales Manager agent that selects and sends the best email from pre-generated drafts."""
    logger.info(f"Starting sales_manager with input: {input_prompt}")
    
    # Step 1: Generate drafts
    tools = [agent1_tool, agent2_tool, agent3_tool]
    drafts = []
    for tool in tools:
        try:
            result = tool.invoke(input_prompt)
            drafts.append(result)
        except Exception as e:
            logger.error(f"Error in {tool.name}: {str(e)}")
            drafts.append(f"Error: {str(e)}")
    
    drafts_input = "\n\n---\n\n".join(drafts)
    logger.info(f"Generated drafts: {drafts_input[:100]}...")
    
    # Step 2: Evaluate and send
    prompt = ChatPromptTemplate.from_messages([
        SystemMessagePromptTemplate.from_template(instructions["sales_manager"]),
        HumanMessagePromptTemplate.from_template("Input prompt: {input_prompt}\nDrafts:\n{drafts}")
    ])
    chain = RunnableSequence(prompt | llm.bind_tools([send_email], tool_choice="send_email"))
    try:
        result = chain.invoke({"input_prompt": input_prompt, "drafts": drafts_input})
        logger.info(f"Raw LLM response: {result}")
        
        # Handle tool call
        if result.tool_calls:
            for tool_call in result.tool_calls:
                if tool_call["name"] == "send_email":
                    logger.info(f"Executing send_email tool call with args: {tool_call['args']}")
                    return send_email.invoke(tool_call["args"]["email_content"])
        # Fallback if no tool call or content
        return result.content if result.content else "No output from LLM or no tool call executed."
    except Exception as e:
        logger.error(f"Error in sales_manager chain: {str(e)}")
        return f"Error in sales_manager: {str(e)}"

# Enable LangSmith tracing
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")  # Ensure this is in .env
os.environ["LANGCHAIN_PROJECT"] = "SalesManagerProject"

# Run the Sales Manager
message = "Send a cold sales email addressed to 'Dear CEO'"
result = sales_manager(message)
print("\n=== Sales Manager Result ===\n")
print(result)
print("\n" + "=" * 40 + "\n")

INFO:__main__:Starting sales_manager with input: Send a cold sales email addressed to 'Dear CEO'
INFO:__main__:Running agent1_tool with input: Send a cold sales email addressed to 'Dear CEO'
INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
INFO:__main__:agent1_tool output: Subject: Simplify Your SOC2 Compliance with ComplAI

Dear CEO,

As a leader in your organization, yo...
INFO:__main__:Running agent2_tool with input: Send a cold sales email addressed to 'Dear CEO'
INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
INFO:__main__:agent2_tool output: Subject: Your SOC2 Audit in 5, 4, 3, 2...

Dear CEO,

I hope this email finds you well-rested and no...
INFO:__main__:Running agent3_tool with input: Send a cold sales email addressed to 'Dear CEO'
INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
INFO:__main__:agent3_tool output: Subject: Streamline You


=== Sales Manager Result ===

Email sent successfully: Subject: Simplify Your SOC2 Compliance with ComplA...


